In [ ]:
import asyncio
import json

import instructor
import logfire
import nest_asyncio
from dotenv import load_dotenv
from openai import AsyncOpenAI
from pydantic import BaseModel, Field
from tqdm.asyncio import tqdm_asyncio

nest_asyncio.apply()

load_dotenv(".env")
logfire.configure()


client = AsyncOpenAI(max_retries=3)

logfire.instrument_openai(client)
client = instructor.from_openai(client)


INPUT_OUTPUT_PAIRS_FILE_PATH = "test_queries.jsonl"
EVALUATION_RESULTS_FILE_PATH = "task_evaluation_results.jsonl"

## Pydantic Models


In [2]:
class ValidationReponse(BaseModel):
    chain_of_thought: str = Field(
        description="Comment vas-tu évaluer la réponse générée ?"
    )
    jugement: bool = Field(
        description="En utilisant le raisonnement ci-dessus, indique True si la réponse correspond à la réponse de référence (ground truth)."
    )
    explication: str = Field(
        description="Explique ici le raisonnement derrière ton évaluation. Quels sont les nombres qui ne sont pas pareils? Mentionne les nombres dans la référance et la réponse générée"
    )

In [3]:
def load_jsonl_file(jsonl_file_path):
    data = []
    with open(jsonl_file_path, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))
    return data


jsonl_file_path = INPUT_OUTPUT_PAIRS_FILE_PATH
data = load_jsonl_file(jsonl_file_path)


async def verify_classification(idx, item, client, semaphore):
    async with semaphore:
        query = item["query"]
        ground_truth = item["ground_truth"]
        generated_output = item["generated_output"]
        with logfire.span(f"Question {idx}: {query}"):
            try:
                verification_result: ValidationReponse = (
                    await client.chat.completions.create(
                        model="gpt-4o",
                        response_model=ValidationReponse,
                        messages=[
                            {
                                "role": "system",
                                "content": (
                                    "La tâche consiste à vérifier si la réponse générée correspond à la réponse de référence (ground truth)."
                                    "Tant que la réponse générée contient la ou les bonnes valeurs présentes dans la réponse de référence, la réponse est considérée comme correcte."
                                    "Les nombres mentionnés dans la réponse générée doivent être exactement identiques à ceux de la réponse de référence."
                                    "Le format de la réponse n'est pas à considérer, tant que les valeurs sont identiques."
                                    "Mentionne les nombres de la réponse de référance et générée dans ta résponse"
                                ),
                            },
                            {
                                "role": "user",
                                "content": (
                                    f"Voici la question originale :\n{query}\n\n"
                                    f"Maintenant, vérifie si la réponse générée est correcte et répond à la question.\n"
                                    f"Voici la réponse de référance :\n{ground_truth}\n\n"
                                    f"et voici la réponse générée :\n{generated_output}\n\n"
                                ),
                            },
                        ],
                        max_tokens=8000,
                        temperature=0,
                    )
                )
            except Exception as e:
                logfire.error(f"Error processing input example {idx}: {e}")

            return idx, verification_result

In [4]:
async def main():
    semaphore = asyncio.Semaphore(100)
    tasks = [
        verify_classification(idx, item, client, semaphore)
        for idx, item in enumerate(data, start=1)
    ]

    results = []
    for future in tqdm_asyncio.as_completed(tasks, total=len(tasks)):
        idx, verification = await future
        results.append((idx, verification))

    results.sort(key=lambda x: x[0])

    verification_file_path = EVALUATION_RESULTS_FILE_PATH
    with open(verification_file_path, "w", encoding="utf-8") as f:
        for idx, verification in results:
            json_line = json.dumps(
                {
                    "index": idx,
                    "evaluation": verification.model_dump(),
                },
                ensure_ascii=False,
            )
            f.write(json_line + "\n")

In [ ]:
await main()

In [ ]:
# Function to load evaluation results from the JSONL file
def load_evaluation_results(jsonl_file_path):
    evaluation_results = []
    with open(jsonl_file_path, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip():
                data = json.loads(line)
                evaluation_results.append(data)
    return evaluation_results


# Load the evaluation results
evaluation_file_path = EVALUATION_RESULTS_FILE_PATH
evaluation_results = load_evaluation_results(evaluation_file_path)


# Count true/false judgments and calculate accuracy
def calculate_accuracy(evaluation_results):
    total_evaluations = len(evaluation_results)
    true_count = sum(
        1
        for result in evaluation_results
        if result.get("evaluation", {}).get("jugement") == True
    )
    false_count = total_evaluations - true_count

    accuracy = true_count / total_evaluations if total_evaluations > 0 else 0

    return {
        "total_evaluations": total_evaluations,
        "true_count": true_count,
        "false_count": false_count,
        "accuracy": accuracy,
    }


# Calculate and display the accuracy metrics
accuracy_metrics = calculate_accuracy(evaluation_results)
print(f"Total evaluations: {accuracy_metrics['total_evaluations']}")
print(f"Correct answers (True): {accuracy_metrics['true_count']}")
print(f"Incorrect answers (False): {accuracy_metrics['false_count']}")
print(f"Accuracy: {accuracy_metrics['accuracy']:.2%}")


# Display the questions the system got wrong
print("\n" + "=" * 80)
print("INCORRECT ANSWERS (jugement=False)".center(80))
print("=" * 80 + "\n")

incorrect_answers = [
    result
    for result in evaluation_results
    if result.get("evaluation", {}).get("jugement") == False
]

if incorrect_answers:
    for i, result in enumerate(incorrect_answers, 1):
        index = result.get("index", "Unknown")
        explication = result.get("evaluation", {}).get(
            "explication", "No reasoning provided"
        )

        print(f"❌ INCORRECT ANSWER #{i} (Index: {index})")
        print("-" * 80)
        print(f"explication:")
        print(f"{explication}")
        print("\n" + "=" * 80 + "\n")
else:
    print("No incorrect answers found!")